In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv',sep='\t')

In [7]:
df

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,8
5568,ham,Will ü b going to esplanade fr home?,36,1
5569,ham,"Pity, * was in mood for that. So...any other s...",57,7
5570,ham,The guy did some bitching but I acted like i'd...,125,1


In [8]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [10]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

### SPlit data into training and test set

In [12]:
from sklearn.model_selection import train_test_split
X = df['message']
y = df['label']

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

### Count vectorizer

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

#### Fit the Vectorizer to the data(Build a vocabulary, count the number of words...)


#### Transform the original tezt message to the vector


In [22]:
X_train_counts = count_vect.fit_transform(X_train)

In [26]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

### Tfidf Transformer

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

In [32]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [33]:
X_train_tfidf.shape

(3733, 7082)

#### Tfidf vectorization combines two previous step of Count_vectorizer and Transformer into one single step

## Tfidf VectoriZer

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [35]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [36]:
X_train_tfidf

<3733x7082 sparse matrix of type '<class 'numpy.float64'>'
	with 49992 stored elements in Compressed Sparse Row format>

#### Train the classifier

In [37]:
from sklearn.svm import LinearSVC
clf = LinearSVC()

In [38]:
clf.fit(X_train_tfidf,y_train)

LinearSVC()

### Combine everything above in a single pipeline

In [41]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [42]:
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [43]:
predictions = text_clf.predict(X_test)

In [44]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [45]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.989668297988037

In [48]:
text_clf.predict(["Hi. How are you doing today?"])

array(['ham'], dtype=object)

In [51]:
text_clf.predict(["Buy the product!!! Get T-shirt Free!!!"])

array(['ham'], dtype=object)

In [ ]:
! git add Text_Feature_Extraction.ipynb
! git commit -m "20:46/28-01-2022"
! git push origin main